<a href="https://colab.research.google.com/github/amitnits2016/Keras/blob/master/RetinaNet/Training_retinanet_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/fizyr/keras-retinanet.git

fatal: destination path 'keras-retinanet' already exists and is not an empty directory.


In [2]:
cd keras-retinanet/

/content/keras-retinanet


In [3]:
!python3 setup.py build install

running build
running build_py
running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
running install
running bdist_egg
running egg_info
writing keras_retinanet.egg-info/PKG-INFO
writing dependency_links to keras_retinanet.egg-info/dependency_links.txt
writing entry points to keras_retinanet.egg-info/entry_points.txt
writing requirements to keras_retinanet.egg-info/requires.txt
writing top-level names to keras_retinanet.egg-info/top_level.txt
writing manifest file 'keras_retinanet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/keras_retinanet
creating build/bdist.linux-x86_64/egg/keras_retinanet/callbacks
copying build/lib.linux-x86_64-3.6/keras_retinanet/callbacks/coco.py -> build/bdist.linux-x86_64/egg/keras_retinanet/callbacks
copying build/lib.linux-x86_64-3.6/keras_retinanet/callbacks/__init__.py -> build/bdist.l

In [4]:
ls

annotations.csv  images/                    setup.cfg
build/           keras_retinanet/           setup.py
classes.csv      keras-retinanet/           snapshots/
CONTRIBUTORS.md  keras_retinanet.egg-info/  tests/
data_load.py     LICENSE                    val_annotations.csv
dist/            README.md
examples/        requirements.txt


In [5]:
# %load /content/drive/My Drive/RetinaNet/data_load.py

In [6]:
# %pycat /content/drive/My Drive/RetinaNet/data_load.py

In [7]:
%%writefile data_load.py
import os
import sys
import math
import glob
import argparse
import xml.etree.ElementTree as ET

# file locations
train_data = '/content/drive/My Drive/RetinaNet/Data/Train/'
test_data = '/content/drive/My Drive/RetinaNet/Data/Test/'

# output
train_annotations = './annotations.csv'
val_annotations = './val_annotations.csv'
label_file = './classes.csv'


# CONVERT the XML annotations to CSV format
def convert_annotation(train_data, test_data, image_id, filename, classes, train=False):
    if train:
        in_file = open(train_data + '%s.xml' % image_id)
    else:
        in_file = open(test_data + '%s.xml' % image_id)
    out_file = open(filename, 'a')
    tree = ET.parse(in_file)
    root = tree.getroot()

    if root.iter('object') is not None:
        for obj in root.iter('object'):
            cls = obj.find('name').text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)

            xmlbox = obj.find('bndbox')
            x1 = math.ceil(float(xmlbox.find('xmin').text))
            y1 = math.ceil(float(xmlbox.find('ymin').text))
            x2 = math.ceil(float(xmlbox.find('xmax').text))
            y2 = math.ceil(float(xmlbox.find('ymax').text))
            if x1 == x2 or y1 == y2:
                continue
            if train:
                out_file.write(
                    f'{train_data + image_id}.jpg,{x1},{y1},{x2},{y2},{cls}\n')
            else:
                out_file.write(
                    f'{test_data + image_id}.jpg,{x1},{y1},{x2},{y2},{cls}\n')
    else:
        if train:
            out_file.write(f'{train_data + image_id}.jpg,,,,,\n')
        else:
            out_file.write(f'{test_data + image_id}.jpg,,,,,\n')


def parse_args(args):
    """ Parse the arguments.
    """
    parser     = argparse.ArgumentParser(description='Simple training script for training a RetinaNet network.')
    # subparsers = parser.add_subparsers(help='Arguments for specific dataset types.', dest='dataset_type')
    # subparsers.required = True

    # group = parser.add_mutually_exclusive_group()
    parser.add_argument('--train', help='Input dir with train images', default=' ', dest='train')
    parser.add_argument('--test',  help='Input dir for test images', default='', dest='test')

    return parser.parse_args(args)


def main(args=None):
    # parse arguments

    if args is None:
        args = sys.argv[1:]
    args = parse_args(args)

    train_data = args.train
    test_data = args.test

    # filter out the train test jpg files
    train_image = glob.glob(train_data + '/*.jpg')

    test_image = glob.glob(test_data + '/*.jpg')

    # define the classes for the model
    labels = ['chair', 'table', 'vase']

    # create new annotation files for  Train/validation
    open(train_annotations, 'w')
    open(val_annotations, 'w')

    train_ids = [os.path.basename(i[:-4]) for i in train_image]
    for image_id in train_ids:
        convert_annotation(train_data, test_data, image_id, train_annotations, labels, train=True)

    val_ids = [os.path.basename(i[:-4]) for i in test_image]

    for image_id in val_ids:
        convert_annotation(train_data, test_data, image_id, val_annotations, labels)

    # creating classes file listing all classes
    with open(label_file, 'w') as f:
        for i, line in enumerate(labels):
            f.write('{},{}\n'.format(line, i))

    print('DONE')


if __name__ == '__main__':
    main()

Overwriting data_load.py


In [8]:
# %rm /content/drive/'My Drive'/RetinaNet/data_load.py

In [9]:
!python3 data_load.py --train '/content/drive/My Drive/RetinaNet/Data/Train/' --test '/content/drive/My Drive/RetinaNet/Data/Test/'

Traceback (most recent call last):
  File "data_load.py", line 109, in <module>
    main()
  File "data_load.py", line 98, in main
    convert_annotation(train_data, test_data, image_id, val_annotations, labels)
  File "data_load.py", line 23, in convert_annotation
    in_file = open(test_data + '%s.xml' % image_id)
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/RetinaNet/Data/Test/Table17.xml'


In [10]:
!python /content/keras-retinanet/keras_retinanet/bin/train.py --backbone='resnet50' --weights /content/keras-retinanet/snapshots/resnet50_coco_best_v2.0.1.h5 --random-transform --gpu=0 --batch-size=4 --steps 800 --epochs 2 --image-min-side=500 --image-max-side=500 --lr 0.001 --weighted-average --compute-val-loss csv /content/keras-retinanet/annotations.csv /content/keras-retinanet/classes.csv \
--val-annotations /content/keras-retinanet/val_annotations.csv

Using TensorFlow backend.
2020-07-30 11:12:03.946366: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-30 11:12:05.874905: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-30 11:12:05.899865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-30 11:12:05.900608: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-30 11:12:05.900678: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-30 11:12:05.902909: I tensorflow/str

In [12]:
!python /content/keras-retinanet/keras_retinanet/bin/train.py --backbone='resnet50' --weights /content/keras-retinanet/snapshots/resnet50_coco_best_v2.0.1.h5 --random-transform --gpu=0 --batch-size=8 --steps 800 --epochs 15 --image-min-side=500 --image-max-side=500 --lr 0.001 --weighted-average --compute-val-loss csv /content/keras-retinanet/annotations.csv /content/keras-retinanet/classes.csv \
--val-annotations /content/keras-retinanet/val_annotations.csv

Using TensorFlow backend.
2020-07-30 11:39:12.486567: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-30 11:39:14.412927: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-30 11:39:14.436588: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-30 11:39:14.437334: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla K80 computeCapability: 3.7
coreClock: 0.8235GHz coreCount: 13 deviceMemorySize: 11.17GiB deviceMemoryBandwidth: 223.96GiB/s
2020-07-30 11:39:14.437385: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-30 11:39:14.439211: I tensorflow/str